In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from itertools import combinations
from itertools import product
from scipy.stats import chi2_contingency

In [1]:
data = pd.read_csv("/kaggle/input/employee-dataset.csv")
data

NameError: name 'pd' is not defined

In [ ]:
#Lets check for duplicates 
data.duplicated().sum()

In [ ]:
#lets remove the duplicates 
data.drop_duplicates(inplace=True) #inplace=True modifies the original dataframe

In [ ]:
#Lets check for null/nan/missing values in the data 
data.isnull().sum()

In [ ]:
data.shape

 So only 2 columns education and previous_yr_rating are having missing values (few in number)

In [ ]:
data.nunique()

In [ ]:
data.info()

In [ ]:
data.describe() # prints the statistics of the numerical columns in the data

# UNIVARIATE Analysis

In [ ]:
data.columns

In [ ]:
num_cols = [col for col in data.columns if data[col].dtypes=="int64" or  data[col].dtypes=="float64"]
num_cols

## Numerical Columns Analysis

1] no_of_trainings

In [ ]:
data["no_of_trainings"].value_counts()

In [ ]:
# it has numeric datatype but has only 9 unique values so can be treated as categorical column
sns.histplot(data=data,x="no_of_trainings",color="cyan")

We can see discrete bars due to the fact that no_of_trainings has only 9 discrete unique values 

Inference : Most of the employee have only 1 or 2 training session with a maximum of 9

2] AGE

In [ ]:
sns.histplot(data=data,x="age",bins=20,color="pink",kde=True)

We can see that most of the employees are youth, with few being aged.  
We have a peak around 30 years age which is quite logical in most of the companies to have young employees with a little work experience

3] length_of_service

In [ ]:
sns.histplot(data=data,x="length_of_service",bins=15,color="green",kde=True)

We can see that most of the employees are dont have very less large length_of_service with their previous employer as the kde is highly right skewed

4] KPIs_met_more_than_80

In [ ]:
data.KPIs_met_more_than_80.unique()

In [ ]:
sns.countplot(data=data,x="KPIs_met_more_than_80",palette=["red","green"])

In [ ]:
counts = data["KPIs_met_more_than_80"].value_counts()
plt.figure(figsize=(8, 8))  # Optional: Set the figure size
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', colors=["red", "green"])
plt.title('Distribution of KPIs Met More Than 80%')
plt.show()

We can see that 64% of the employees havent achieved Key Performance Indicators which is a concern for the company managers.  
They have to work on improving this %age upto 70% and even more for growth of the company

5] AWARDS WON

In [ ]:
counts = data["awards_won"].value_counts()
plt.figure(figsize=(8, 8))  # Optional: Set the figure size
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', colors=["red", "green"])
plt.title('awards_won')
plt.show()

Only 2.3% of the employees have won the award which is quite logical that award is given by the company to only few cream employees for there excelent performance

6] avg_training_score

In [ ]:
sns.histplot(data=data,x="avg_training_score",bins=25,kde=True,color="orange")

We can see a wide variation in the avg_training_score range.  Less employees have scored above 80

7] previous_year_rating

In [ ]:
sns.countplot(data=data,x="previous_year_rating")

## Categorical Column Analysis

In [ ]:
cat_cols = [col for col in data.columns if data[col].dtypes=="object"]
cat_cols

1] Department

In [ ]:
data.department.value_counts()

In [ ]:
sns.countplot(data=data,y="department")

In [ ]:
counts = data["department"].value_counts()
plt.figure(figsize=(8, 8))  # Optional: Set the figure size
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', colors=["red", "green","purple"])
plt.title('department')
plt.show()

Operations and Sales&Marketing covers half of company employees with very few in R&D, Legal, Finance etc

2] Education

In [ ]:
sns.countplot(data=data,x="education",color="yellow")

Max employees are decently educated, few being highly educated, very less being less educated

3] Gender

In [ ]:
counts = data["gender"].value_counts()
plt.figure(figsize=(8, 8))  # Optional: Set the figure size
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', colors=["green", "cyan"])
plt.title('gender')
plt.show()

More than 70% employees in the company are male

4] recruitment_channel

In [ ]:
counts = data["recruitment_channel"].value_counts()
plt.figure(figsize=(8, 8))  # Optional: Set the figure size
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', colors=["blue", "red","green"])
plt.title('recruitment_channel')
plt.show()

5] Region

In [ ]:
sns.countplot(data=data,y="region",color="yellow")

Max employees belong to region2, while remaining approx evenly spread in rest locations

## Here we have completed analysis of indivisual column, based on this lets fill the null values in the dataset

Based on the analysis of previous_year_rating, i think its better to fill nan values using meadian (which is also the mode)

In [ ]:
si1 = SimpleImputer(strategy="median")
imputed_values = si1.fit_transform(data[["previous_year_rating"]])
# Assign the imputed values back to the original DataFrame
data["previous_year_rating"] = imputed_values

For the education column lets use the mode (graduate) as its categorical data mean, meadin doesnt make sense and also graduate class is largely dominant over the other two

In [ ]:
# Lets use fillna this time
mode = data['education'].mode()[0]
data['education'] = data['education'].fillna(mode)

In [ ]:
data.isnull().sum()

# Now we have a completed the missing value imputation part

# BIVARIATE ANALYSIS

# 1] Numerical and Numerical

In [ ]:
numerical_data = data.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numerical_data.corr()
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='viridis', square=True, cbar=True, linewidths=0.5)
plt.title('Heatmap of Correlation Matrix for Numerical Data')
plt.show()

From the heatmap of correlation matrix we can see that ther isnt much linear relationship between the maximum numerical columns

Two things that can be infered are  
1] length of service is quite well correlated with age which is quite logical as person with less age cant be expected to have large experience.  
2] KPI more than 80 is little related with the prev_yr_rating.

## 2] Categorical and categorical

In [ ]:
# lets not plot for region as its giving very dirty graphs which cant be used to infer anything
cat_col = cat_cols.copy()
cat_col.remove("region")
for col1, col2 in combinations(cat_col, 2):
    plt.figure(figsize=(10, 6))
    sns.countplot(data=data, y=col1, hue=col2, palette='Set1')
    plt.title(f'Count of {col1} by {col2}')
    plt.xlabel(col1)
    plt.ylabel('Count')
    plt.legend(title=col2)
    plt.show()

We have plotted count plots for various combinations of categorical columns

In [ ]:
for col1, col2 in combinations(cat_cols, 2):
    # Create contingency table
    contingency_table = pd.crosstab(data[col1], data[col2])
    # Perform Chi-Square test
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    print(f"Chi-Square Test between {col1} and {col2}:")
    print(f"Chi2 Statistic: {chi2:.4f}, p-value: {p:.4f}, Degrees of Freedom: {dof}")

high p-value implies there is no association between the two categorical data¶

So from the above result we can say the following things:
1] department and region, department and education, department and gender, department and recruitment_channel have a relationship between them
ie. department is dependent on all these factors 2] region is also dependent on all other cat factors education, gender, department, recruitment_channel
3] only indepedent pair from the observation is gender and recruitment channel

## 3] Numerical and Categorical

In [ ]:
for num_col, cat_col in product(num_cols, cat_cols):
    # Calculate mean and standard deviation for the numerical column based on categorical variable
    summary = data.groupby(cat_col).agg(
        mean_value=(num_col, 'mean'),
        std_value=(num_col, 'std'),
        count=(num_col, 'size')
    ).reset_index()

    # Calculate standard error for error bars
    summary['std_error'] = summary['std_value'] / summary['count']**0.5

    # Plotting line chart with error bars
    plt.figure(figsize=(12, 6))
    sns.lineplot(data=summary, x=cat_col, y='mean_value', marker='o')

    # Add error bars
    plt.errorbar(x=summary[cat_col], 
                 y=summary['mean_value'], 
                 yerr=summary['std_error'], 
                 fmt='none', 
                 c='black', 
                 capsize=5)

    # Adding titles and labels
    plt.title(f'Mean {num_col} by {cat_col}')
    plt.xlabel(cat_col)
    plt.ylabel(f'Mean {num_col}')
    plt.grid()

    # Show the plot
    plt.show()

## Scroll the above graphs for all possible combos¶

In [ ]:
# Constants
SAMPLE_SIZE_THRESHOLD = 30
SIGNIFICANCE_LEVEL = 0.05

# Assuming you have two lists: cat_cols (categorical) and num_cols (numerical)
# Replace `data` with your actual DataFrame
for cat_col, num_col in product(cat_cols, num_cols):
    # Drop any rows with missing values for the current pair of columns
    filtered_data = data[[cat_col, num_col]].dropna()

    # Group numerical data by the categorical column
    groups = filtered_data.groupby(cat_col)[num_col]

    # Get unique categories
    unique_categories = filtered_data[cat_col].unique()

    # Skip if there's only one category (nothing to compare)
    if len(unique_categories) < 2:
        print(f"Skipping {cat_col} - {num_col}: Less than two categories")
        continue

    # Handle binary categorical columns (T-test)
    if len(unique_categories) == 2:
        group1 = groups.get_group(unique_categories[0])
        group2 = groups.get_group(unique_categories[1])

        # Check which test to perform based on sample size
        if len(group1) > SAMPLE_SIZE_THRESHOLD and len(group2) > SAMPLE_SIZE_THRESHOLD:
            # Perform Z-test approximation using T-test
            stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
            test_type = "Z-test (T-test approximation)"
        else:
            # Perform T-test
            stat, p_value = stats.ttest_ind(group1, group2, equal_var=False)
            test_type = "T-test"
        
        # Print results
        print(f"{test_type} between {cat_col} and {num_col}:")
        print(f"Statistic: {stat:.4f}, p-value: {p_value:.4f}")

        # Significance test
        if p_value < SIGNIFICANCE_LEVEL:
            print(f"Reject the null hypothesis (p-value < {SIGNIFICANCE_LEVEL})")
        else:
            print(f"Fail to reject the null hypothesis (p-value >= {SIGNIFICANCE_LEVEL})")

    # Handle categorical columns with more than 2 categories (ANOVA)
    else:
        # Create a list of numerical data for each category
        category_data = [group for _, group in groups]

        # Perform one-way ANOVA
        stat, p_value = stats.f_oneway(*category_data)
        test_type = "ANOVA"

        # Print results
        print(f"{test_type} between {cat_col} and {num_col}:")
        print(f"Statistic: {stat:.4f}, p-value: {p_value:.4f}")

        # Significance test
        if p_value < SIGNIFICANCE_LEVEL:
            print(f"Reject the null hypothesis (p-value < {SIGNIFICANCE_LEVEL})")
        else:
            print(f"Fail to reject the null hypothesis (p-value >= {SIGNIFICANCE_LEVEL})")

    print("\n")

## Conclusions

1] The company has to work on efficiency of the employees as less employees are able to meet kpi more than 80  
2] Higher education doesnt necesarily imply the employee is better as we can see from kpi met as well as awards won  
3] The company can work on improving female:male ratio  
4] Sales and marketing is major sector for concern which has least avg in meeting the kpi of 80 which must be improved to inc the revenue  
5] There is high dependency between columns so before training a model do proper feature engineering for better results.  

# Cleaning

In [ ]:
# lets check for outliers
sns.boxplot(data=data,x="length_of_service")

In [ ]:
# Calculate Q1 (25th percentile) and Q3 (75th percentile)
Q1 = data['length_of_service'].quantile(0.25)
Q3 = data['length_of_service'].quantile(0.75)
IQR = Q3 - Q1

# Define lower and upper bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter the DataFrame to remove outliers
data_filtered = data[(data['length_of_service'] >= lower_bound) & (data['length_of_service'] <= upper_bound)]

# Optionally, print the number of rows before and after
print(f'Original data size: {data.shape[0]}')
print(f'Filtered data size: {data_filtered.shape[0]}')

SO we have removed 1028 outliers in the data based on lenght_of_service column

# Encoding Categorical Data

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
le_department = LabelEncoder()
le_region = LabelEncoder()

# Apply Label Encoding to the 'department' column
data['department_encoded'] = le_department.fit_transform(data['department'])

# Apply Label Encoding to the 'region' column
data['region_encoded'] = le_region.fit_transform(data['region'])
# Drop the original categorical columns
data = data.drop(columns=['department', 'region'])
# Display the resulting DataFrame
print(data)

In [ ]:
# one-hot encoding
data_encoded = pd.get_dummies(data, columns=['gender', 'recruitment_channel'], drop_first=True) 
# This could have also be done using sklearn
# Drop the original categorical columns
data = data.drop(columns=['gender','recruitment_channel'])
# Display the resulting DataFrame
print(data_encoded)

In [ ]:
data.education.unique()

In [ ]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

# Define the order of education levels (from lowest to highest)
education_levels = ['Below Secondary','Bachelors', 'Masters & above']
# Initialize the OrdinalEncoder with the specified categories
ordinal_encoder = OrdinalEncoder(categories=[education_levels])
# Apply ordinal encoding to the education column
data_encoded['education_encoded'] = ordinal_encoder.fit_transform(data_encoded[['education']])
data_encoded.drop(columns=["education"],inplace=True)
# Display the DataFrame with the encoded column
print(data_encoded)

# now our data is ready to be passsed to ML model.

# For better results, do proper feature engineering